# Batch Processing

This tutorial demonstrates how to use Kluster.ai's batch API to process multiple tasks. 

You'll learn how to **create, submit, monitor**, and **retrieve** results from batch jobs.

## Setup and Prerequisites

First, let's set up our environment with the necessary libraries:

In [ ]:
# Install required libraries
%pip install openai pandas tqdm jupyterlab ipywidgets

In [8]:
# Import libraries
import os
import json
import time
import pandas as pd
import numpy as np
from getpass import getpass
from openai import OpenAI
from tqdm.notebook import tqdm
from IPython.display import display, Markdown, clear_output

### Configuration Parameters

Set the model and batch size parameters below to configure your batch job:

In [ ]:
# User-configurable parameters

# Choose your model - options include:
# - google/gemma-3-27b-it 
# - meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8 
# - meta-llama/Llama-4-Scout-17B-16E-Instruct 
# - klusterai/Qwen2.5-7B-Instruct-Turbo 
# - deepseek-ai/DeepSeek-V3-0324 
# - deepseek-ai/DeepSeek-R1 

PRIMARY_MODEL = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"  # Choose a faster model by default

# For certain tasks we might want a separate model (optional)
SECONDARY_MODEL = PRIMARY_MODEL  # Default: use same model for all tasks

# Number of quotes to process (for testing, use a small number)
NUM_QUOTES = 15  # Start with 5 for testing, increase later

### Set up API Keys and Model Endpoints

We'll use getpass to securely input your kluster.ai API key without displaying it.

You can get your API Key from your [kluster.ai Account ](https://platform.kluster.ai/apikeys)

In [ ]:
# Set up Kluster.ai client
api_key = getpass("Enter your kluster.ai API key: ")

# Initialize client pointing to kluster.ai API
client = OpenAI(
    base_url="https://api.kluster.ai/v1",
    api_key=api_key,
)

## Creating a Dataset for the Batch Processing Job

Let's create a batch job that analyzes movie quotes across multiple dimensions:

In [11]:
# Sample movie quotes for analysis
movie_quotes = [
    "May the Force be with you.",
    "I'm going to make him an offer he can't refuse.",
    "You talking to me?",
    "E.T. phone home.",
    "Here's looking at you, kid.",
    "Go ahead, make my day.",
    "Show me the money!",
    "Houston, we have a problem.",
    "I'll be back.",
    "Life is like a box of chocolates, you never know what you're gonna get.",
    "There's no place like home.",
    "I see dead people.",
    "My precious...",
    "To infinity and beyond!",
    "You can't handle the truth!"
]

# Create a DataFrame for better visualization
quotes_df = pd.DataFrame({"quote": movie_quotes})
display(quotes_df)

,quote
0,May the Force be with you.
1,I'm going to make him an offer he can't refuse.
2,You talking to me?
3,E.T. phone home.
4,"Here's looking at you, kid."
5,"Go ahead, make my day."
6,Show me the money!
7,"Houston, we have a problem."
8,I'll be back.
9,"Life is like a box of chocolates, you never kn..."


For each movie quote, we'll generate three tasks:

1. **Sentiment Analysis**: Determine whether the quote expresses a **positive, negative**, or **neutral** sentiment.
2. **Movie Identification**: Identify the movie from which the quote originates.
3. **Creative Follow-up**: Generate a witty or creative follow-up line to the original quote.

By batching these tasks, we can efficiently analyze multiple aspects of each quote **in parallel.**

In [12]:
# Define multiple tasks for each quote
batch_requests = []

for idx, quote in enumerate(movie_quotes):
    # Task 1: Sentiment analysis
    sentiment_request = {
        "custom_id": f"sentiment-{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": PRIMARY_MODEL,
            "messages": [
                {"role": "system", "content": "You are an expert sentiment analyst. Respond with only a single word: POSITIVE, NEGATIVE, or NEUTRAL."},
                {"role": "user", "content": f"Analyze the sentiment of this quote: '{quote}'"}
            ],
            "max_completion_tokens": 15,
        }
    }
    batch_requests.append(sentiment_request)
    
    # Task 2: Identify movie
    movie_request = {
        "custom_id": f"movie-{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": PRIMARY_MODEL,
            "messages": [
                {"role": "system", "content": "You are a movie expert. Name only the movie this quote is from. Just the title, nothing else."},
                {"role": "user", "content": f"What movie is this quote from: '{quote}'"}
            ],
            "max_completion_tokens": 100,
        }
    }
    batch_requests.append(movie_request)
    
    # Task 3: Generate a follow-up line
    followup_request = {
        "custom_id": f"followup-{idx}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": SECONDARY_MODEL,  # Using the secondary model (could be the same)
            "messages": [
                {"role": "system", "content": "You are a creative scriptwriter. Write a creative and witty follow-up line to this movie quote."},
                {"role": "user", "content": f"Write a creative follow-up line to this movie quote: '{quote}'"}
            ],
            "max_completion_tokens": 200,
        }
    }
    batch_requests.append(followup_request)

print(f"Created {len(batch_requests)} batch requests for processing ({NUM_QUOTES} quotes × 3 task types)")

Created 45 batch requests for processing (15 quotes × 3 task types)


## Submitting the Batch Job

Now let's save our requests to a JSONL file and submit the batch job:

In [17]:
# Save tasks to a JSONL file
file_name = "movie_quotes_analysis.jsonl"
with open(file_name, "w") as file:
    for request in batch_requests:
        file.write(json.dumps(request) + "\n")

# Upload batch job file
print("Uploading batch file...")
batch_input_file = client.files.create(
    file=open(file_name, "rb"),
    purpose="batch"
)
print(f"File uploaded with ID: {batch_input_file.id}")

# Submit batch job
print("Submitting batch job...")
batch_request = client.batches.create(
    input_file_id=batch_input_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)
print(f"Batch job submitted with ID: {batch_request.id}")

Uploading batch file...
File uploaded with ID: 6823111ab494ced538c098a8
Submitting batch job...
Batch job submitted with ID: 6823111a1c555419fe3bdcb1


### 3.1 Monitoring Batch Job Progress

Let's create a progress display for our batch job with error handling:

In [20]:
# Monitor batch job progress with simple updates
print("Monitoring batch job progress...")

previous_completed = 0

while True:
    batch_status = client.batches.retrieve(batch_request.id)
    
    # Get counts and calculate progress
    completed = batch_status.request_counts.completed
    total = batch_status.request_counts.total
    
    # Only print when there's a change in completed count
    if completed > previous_completed:
        print(f"Progress: {completed}/{total} tasks completed")
        previous_completed = completed
    
    # Exit conditions
    status = batch_status.status
    print(f"Status: {status}")
    
    if status.lower() in ["completed", "failed", "cancelled"]:
        if status.lower() == "completed":
            print(f"✅ Batch job completed successfully!")
        elif status.lower() == "failed":
            print(f"❌ Batch job failed. Please check the job status manually.")
        else:
            print(f"⚫ Batch job was cancelled.")
        break
        
    # Wait before checking again
    time.sleep(5)

Monitoring batch job progress...
Progress: 45/45 tasks completed
Status: completed
✅ Batch job completed successfully!


## Retrieving and Displaying Results

Once our batch job completes, let's retrieve the results and display a table:

In [19]:
# Retrieve and process batch results
try:
    # Get the latest job status if needed
    batch_status = client.batches.retrieve(batch_request.id)
    
    if batch_status.status.lower() == "completed":
        # Parse results more concisely
        result_content = client.files.content(batch_status.output_file_id).content
        results = [json.loads(line) for line in result_content.decode().strip().split("\n")]
        print(f"Retrieved {len(results)} results successfully")
        
        # Initialize organized data dictionary (same as before)
        organized_data = {idx: {"quote": quote, "sentiment": "", "movie": "", "followup": ""} 
                         for idx, quote in enumerate(movie_quotes)}
        
        # Process results (simplified slightly)
        for result in results:
            custom_id = result.get("custom_id", "")
            if "-" in custom_id:
                task_type, idx_str = custom_id.split("-", 1)
                idx = int(idx_str)
                
                # Get content from nested response
                choices = result.get("response", {}).get("body", {}).get("choices", [])
                if choices:
                    content = choices[0].get("message", {}).get("content", "")
                    if idx < len(movie_quotes):
                        organized_data[idx][task_type] = content
        
        # Create DataFrame (same as before)
        analysis_results = [
            {
                "Quote": data["quote"],
                "Movie": data["movie"],
                "Sentiment": data["sentiment"],
                "Follow-up Line": data["followup"]
            } for idx, data in organized_data.items()
        ]
        
        # Display results
        pd.set_option('display.max_colwidth', None)
        results_df = pd.DataFrame(analysis_results)
        display(results_df)
        
        # Show sentiment distribution
        sentiment_counts = results_df["Sentiment"].value_counts()
        print("\nSentiment Distribution:")
        for sentiment, count in sentiment_counts.items():
            print(f"{sentiment}: {count}")
    else:
        print(f"⚠️ Batch job has status: {batch_status.status}")
except Exception as e:
    print(f"❌ Error retrieving batch results: {str(e)}")

Retrieved 45 results successfully


,Quote,Movie,Sentiment,Follow-up Line
0,May the Force be with you.,Star Wars: Episode IV - A New Hope,POSITIVE,"'But let's be real, you're gonna need a good accountant too, with tax returns that complicated.'"
1,I'm going to make him an offer he can't refuse.,The Godfather,NEGATIVE,"""And if he does refuse, I'll make sure his accountant has a change of heart... and a new address."""
2,You talking to me?,Taxi Driver,NEUTRAL,"""Only if you're talking back, because I'm buying."""
3,E.T. phone home.,E.T. the Extra-Terrestrial,NEUTRAL,"'And hopefully, with a decent data plan, because my spaceship's Wi-Fi is on the fritz!'"
4,"Here's looking at you, kid.",Casablanca,POSITIVE,"""And here's hoping you don't end up like my ex, stuck in a bottle with a cork that's harder to swallow than a Casablanca breakup."""
5,"Go ahead, make my day.",Sudden Impact,NEGATIVE,"""But first, could you hand me that ashtray, I'm dying for a cigarette."""
6,Show me the money!,Jerry Maguire,POSITIVE,"""I'm not asking for a handout, I'm asking for a hand... to count it, because I'm going to need both hands to count it, it's going to be that much!"""
7,"Houston, we have a problem.",Apollo 13,NEGATIVE,"""Well, that's a first – I was hoping for a decent Wi-Fi signal, but I guess 'Houston, we have a router meltdown' just isn't as catchy."""
8,I'll be back.,The Terminator,"THREATENING is not an option, I'll choose NEUTRAL.","'And when I am, you'll be begging to be gone.'"
9,"Life is like a box of chocolates, you never know what you're gonna get.",Forrest Gump,POSITIVE,"'But with my luck, I'm pretty sure I'll get the one with the coconut cream and regret it for the rest of the day.'"



Sentiment Distribution:
POSITIVE: 7
NEGATIVE: 5
NEUTRAL: 2
THREATENING is not an option, I'll choose NEUTRAL.: 1


## Advanced: Listing and Managing Batch Jobs

Let's add a functionality to list and manage our batch jobs:

In [97]:
# List all batch jobs
def list_batch_jobs():
    try:
        response = client.batches.list(limit=10)
        
        # Display as a DataFrame
        jobs_data = []
        for job in response.data:
            jobs_data.append({
                "ID": job.id,
                "Status": job.status,
                "Created": pd.to_datetime(job.created_at, unit='s'),
                "Completed": pd.to_datetime(job.completed_at, unit='s') if job.completed_at else "N/A",
                "Total Requests": job.request_counts.total,
                "Completed Requests": job.request_counts.completed
            })
            
        return pd.DataFrame(jobs_data)
    except Exception as e:
        print(f"Error listing batch jobs: {e}")
        return pd.DataFrame()

# Display recent batch jobs
recent_jobs = list_batch_jobs()
display(Markdown("## Recent Batch Jobs"))
display(recent_jobs)

## Recent Batch Jobs

,ID,Status,Created,Completed,Total Requests,Completed Requests
0,68221787f3098dc792a547f9,completed,2025-05-12 15:45:11,2025-05-12 15:45:14,45,45
1,68221745e5f4b4fb13176a22,completed,2025-05-12 15:44:05,2025-05-12 15:44:14,45,45
2,68221701e5f4b4fb13176714,completed,2025-05-12 15:42:57,2025-05-12 15:43:06,45,45
3,682216156c35fc54cc07705d,completed,2025-05-12 15:39:01,2025-05-12 15:39:04,45,45
4,682215d5cd770dcad800627c,completed,2025-05-12 15:37:57,2025-05-12 15:38:00,45,45
5,68221598e5f4b4fb131755dd,completed,2025-05-12 15:36:56,2025-05-12 15:36:59,45,45
6,68221550f3098dc792a52978,completed,2025-05-12 15:35:44,2025-05-12 15:35:47,45,45
7,6822145badafd73172ea4e4f,completed,2025-05-12 15:31:39,2025-05-12 15:31:42,45,45
8,68221427e5f4b4fb13174539,completed,2025-05-12 15:30:47,2025-05-12 15:30:53,45,45
9,682212cf4e1240bf3c8c59b0,completed,2025-05-12 15:25:03,2025-05-12 15:25:07,45,45


In [ ]:
# Function to cancel a batch job
def cancel_batch_job(batch_id):
    try:
        response = client.batches.cancel(batch_id)
        print(f"Job {batch_id} cancellation requested. New status: {response.status}")
        return response
    except Exception as e:
        print(f"Error cancelling batch job: {e}")
        return None

## Practical Use Cases

The batch processing capabilities you've just learned can be applied to many real-world scenarios:

1. **Content Moderation**: Process thousands of user comments/posts to detect inappropriate content
2. **Customer Support**: Analyze large volumes of support tickets for sentiment, urgency, and categorization
3. **Market Research**: Process survey responses or product reviews at scale
4. **Document Processing**: Extract key information from legal documents, contracts, or reports
5. **Educational Assessment**: Grade and provide feedback on student essays or assignments
6. **Media Analysis**: Analyze news articles, social media posts, or transcripts for trends and insights
7. **Data Enrichment**: Add AI-generated metadata to your existing datasets

With [kluster.ai](https://platform.kluster.ai/batch) batch API, these tasks can be performed efficiently at scale without building complex infrastructure.


## Conclusion

- **Efficiency**: Process thousands of tasks in parallel
- **Cost-effective**: Optimize for throughput rather than latency
- **Flexible**: Use **different models** and prompts within the same batch
- **Scalable**: Handle datasets of any size
- **Easy to use**: Simple integration with the OpenAI SDK